In [1]:
import sys
sys.path.append('..')

import torch
from torch.nn import BCELoss
from torch.optim import Adam
from torchvision.models import resnet18

from lib.models import SimpleClassifier, ResNet, get_resnet
from lib.trainer import Trainer
from lib.utils import train_parse_args, get_data_loaders
from lib.constants import get_hyperparameters

In [4]:
params = get_hyperparameters()
params['batch_size'] = 16

if not params['disable_cuda'] and torch.cuda.is_available():
    params['device'] = torch.device('cuda:0')
else:
    params['device'] = torch.device('cpu')

print(torch.cuda.is_available(), params['device'])

True cuda:0


In [5]:
loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'],
    device=params['device'])

#model = get_resnet()
model = ResNet()
model.cuda()

optimizer = Adam(model.parameters(), lr=3e-4)
criterion = BCELoss()
trainer = Trainer(params, model, optimizer, criterion)

trainer.run(loaders)

EPOCH 0
0 0 200.0
0 50 200.0
0 100 187.5
0 150 187.5
0 200 200.0
0 250 175.0
0 300 200.0
0 350 200.0
0 400 187.5
0 450 187.5
0 500 187.5
0 550 200.0
0 600 200.0
0 650 200.0
0 700 200.0
0 750 200.0
0 800 200.0
0 850 200.0


KeyError: '/home/sergevkim/git/FacesClassification/data/CelebaHQ/0.png'